In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

let's import our libaries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
print('setup complete')

setup complete


# # read the data and display the top 5 rows

In [3]:
df = pd.read_csv('ab_test.csv')
df.head()

,id,time,con_treat,page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1


In [4]:
df.columns

Index(['id', 'time', 'con_treat', 'page', 'converted'], dtype='object')

In [5]:
df.columns = ['user_id','timestamp','group','landing_page','converted']

In [6]:
print(f'Numbet of rows: {df.shape[0]}')
print(f'Numbet of unique users: {df.user_id.nunique()}')



Numbet of rows: 294478
Numbet of unique users: 290584


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [8]:
# to check missing values
df.isna().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

In [9]:
# to find out group value with teatment
n_treat=df[df['group']=='treatment'].shape[0]

In [10]:
# to find out new pages
n_new_page=df[df['landing_page']=='new_page'].shape[0]

In [11]:
# difference between treatment and newpage
print('difference: ',n_treat-n_new_page)

difference:  37


there is mismatch between number of users assigned to treatment and the number of those landed on treatment page. this might indicate a problem with the data and needs furthers exploration.

In [12]:
# ok let's see that rows
df[(df['group']=='treatment') & (df['landing_page']=='old_page')]

,user_id,timestamp,group,landing_page,converted
308,857184,34:59.8,treatment,old_page,0
327,686623,26:40.7,treatment,old_page,0
357,856078,29:30.4,treatment,old_page,0
685,666385,11:54.8,treatment,old_page,0
713,748761,47:44.4,treatment,old_page,0
...,...,...,...,...,...
293773,688144,34:50.5,treatment,old_page,1
293817,876037,15:09.0,treatment,old_page,1
293917,738357,37:55.7,treatment,old_page,0
294014,813406,25:33.2,treatment,old_page,0


In [13]:
df_missmatch=df[(df['group']=='treatment') & (df['landing_page']=='old_page') | (df['group']=='control') & (df['landing_page']=='new_page')]

In [14]:
print('the number of missmatch rows: ',df_missmatch.shape[0])

the number of missmatch rows:  3893


In [15]:
print('the percentage of missmatch rows: ',round(df_missmatch.shape[0] / len(df)*100,2),'%')

the percentage of missmatch rows:  1.32 %


As you can see, there are 3893 rows where treatment does not match with new_page or control does not match with old_page, we can not be suer that if this row truly received the new or old page. How we will deal with those rows?

In [16]:
df2 = df[(df["group"] == "treatment") & (df["landing_page"] == "new_page")
        |(df["group"] == "control") & (df["landing_page"] == "old_page")]

len(df2)

290585

In [17]:
df2.head()

,user_id,timestamp,group,landing_page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1


In [18]:
# Double Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

0

In [19]:
# Another way to double Check all of the correct rows were removed 
df_mismatch = df2[(df2["group"] == "treatment") & (df2["landing_page"] == "old_page")
               |(df2["group"] == "control") & (df2["landing_page"] == "new_page")]

In [20]:
percent_mismatch = round(df_mismatch.shape[0] / len(df2) * 100, 2)
print(f'Number of mismatched rows: {percent_mismatch} rows')
print(f'Percent of mismatched rows: {percent_mismatch} percent')

Number of mismatched rows: 0.0 rows
Percent of mismatched rows: 0.0 percent


In [21]:
# unique user id in df3
df2.user_id.nunique()

290584

In [22]:
# number of repeated id's in df2
len(df2) - df2.user_id.nunique()

1

In [23]:
# display the duplicated row
df2[df2.duplicated('user_id')==True]

,user_id,timestamp,group,landing_page,converted
2893,773192,55:59.6,treatment,new_page,0


In [24]:
# now we drop duplicated row
df2=df2.drop_duplicates('user_id')

# # Probability

In [25]:
# percentage of convergance
df2.converted.mean()*100

11.959708724499627

In [26]:
df2.user_id = df2.user_id.astype(str)
df2.groupby('group').mean()*100

,converted
group,
control,12.038630
treatment,11.880807


In [27]:
# probability according to individual received the new page
df2.landing_page.value_counts()

new_page    145310
old_page    145274
Name: landing_page, dtype: int64

# # A/B Testing 

In [ ]:
#creating the smapling distribution of difference in means
means_diff = []
size = df.shape[0]
for _ in range(10000):
    sample = df2.sample(size, replace = True)
    control_mean = sample[sample["group"] == "control"]["converted"].mean()
    treat_mean = sample[sample["group"] == "treatment"]["converted"].mean()
    means_diff.append(treat_mean - control_mean)

In [ ]:
plt.figure(figsize=(8,4),dpi=100)
sns.hist(means_df,bins=30)
